In [10]:
# 필요한 라이브러리 가져오기
import cv2
import mediapipe as mp
import csv
import os

# Mediapipe의 Holistic 모델 로드
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils
video_id = 'Z76'

# 입력
video_path = f'C:/Users/jk/action_assess_3_chart/data/video/{video_id}/313-2-1-15-{video_id}_D.avi'
# 출력
csv_path = f'C:/Users/jk/action_assess_3_chart/data/output/csv_key/{video_id}/3313-2-1-15-{video_id}_D_keypoints.csv'
frames_output_folder = f'C:/Users/jk/action_assess_3_chart/data/output/frame_key/{video_id}/313-2-1-15-{video_id}_D/'

# 폴더 생성
os.makedirs(os.path.dirname(csv_path), exist_ok=True)
os.makedirs(frames_output_folder, exist_ok=True)

# 비디오 스트림 열기
cap = cv2.VideoCapture(video_path)

# 비디오 저장 설정
fourcc = cv2.VideoWriter_fourcc(*'XVID')

# CSV 파일 열기
with open(csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # CSV 헤더 작성
    header = ['frame']
    for i in range(33):
        header.extend([f'x_{i}', f'y_{i}', f'z_{i}', f'visibility_{i}'])
    for i in range(33): 
        header.extend([f'world_x_{i}', f'world_y_{i}', f'world_z_{i}', f'world_visibility_{i}'])
    writer.writerow(header)

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # 프레임을 RGB로 변환
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Mediapipe를 사용하여 키포인트 추출
        results = pose.process(image)
        
        # 키포인트를 그리기
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            # 키포인트를 CSV 파일에 저장
            row = [frame_idx]
            for landmark in results.pose_landmarks.landmark:
                row.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
            for landmark in results.pose_world_landmarks.landmark:
                row.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
            writer.writerow(row)
        
        # 결과를 출력
        #cv2.imshow('Frame', frame)
        
        # 프레임을 이미지 파일로 저장
        frame_filename = os.path.join(frames_output_folder, f'frame_{frame_idx:04d}.png')
        cv2.imwrite(frame_filename, frame)
        
        frame_idx += 1
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()